In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')

In [ ]:
items.head()

In [ ]:
item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')

In [ ]:
item_categories.head()

In [ ]:
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')

In [ ]:
shops.head()

In [ ]:
train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
Id = test['ID']
test.head()

In [ ]:
test.shape

In [ ]:
train.info()

In [ ]:
train['date'] = pd.to_datetime(train['date'])

In [ ]:
train.isna().sum()

In [ ]:
train['date'] = train['date'].apply(lambda x: x.strftime("%Y-%m"))

In [ ]:
train.head().sort_values(by = 'date')

In [ ]:
train.drop(['date_block_num','item_price'] , axis =1, inplace= True)

In [ ]:
train.head().sort_values(by = 'date')

In [ ]:
df = train.groupby(['date','shop_id','item_id']).sum()
df

In [ ]:
df = train.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
df.reset_index(inplace=True)
df.head()

In [ ]:
test_df = pd.merge(test, df, on=['shop_id','item_id'], how='left')
# test_df.drop(['ID', '2013-01'], axis=1, inplace=True)
test_df = test_df.fillna(0)

In [ ]:
test_df.head()

In [ ]:
train_df = df

In [ ]:
Y = df['2015-10'].values
X = df.drop(['2015-10'], axis = 1)
test_full = test_df

from sklearn.model_selection import train_test_split

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, Y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = test_df[my_cols].copy()

In [ ]:
input_shape = X_train.shape

# ReLU - Model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

keras.backend.clear_session()


model = keras.models.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

model.compile(loss='mean_absolute_error',
            optimizer=keras.optimizers.Adam(0.001))

In [ ]:
keras.backend.clear_session()

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=3, # how many epochs to wait before stopping
    restore_best_weights=True,
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=64,
    epochs=500,
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=1,  # turn off training log
)
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print(("Minimum Validation Loss: {:0.4f}").format(history_df['val_loss'].min()))

In [ ]:
prediction = model.predict(X_test)

In [ ]:
prediction

In [ ]:
#prediction = list(map(round, prediction))

In [ ]:
sample_sub = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
print(sample_sub.shape)
sample_sub.head()

In [ ]:
sample_sub['item_cnt_month'] = prediction
sample_sub.to_csv('prediction.csv', index=False)
sample_sub.head()

# LSTM - Model

In [ ]:
x_train = X_train.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_valid = X_valid.to_numpy()
x_valid = np.reshape(x_valid, (x_valid.shape[0], x_valid.shape[1], 1))
x_test = X_test.to_numpy()
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

keras.backend.clear_session()


model = keras.models.Sequential([
    layers.LSTM(150, return_sequences=True, input_shape=(x_train.shape[1], 1)),
    layers.LSTM(50, return_sequences=True),
    layers.LSTM(20, return_sequences=False),
    layers.Dense(1)
  ])

model.compile(loss='mean_absolute_error',
            optimizer=keras.optimizers.Adam(0.001))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy']) # we want to minimize loss

In [ ]:
keras.backend.clear_session()

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=3, # how many epochs to wait before stopping
    restore_best_weights=True,
)

history = model.fit(
    x_train, y_train,
    validation_data=(x_valid, y_valid),
    batch_size=1024,
    epochs=5,
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=1,  # turn off training log
)
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print(("Minimum Validation Loss: {:0.4f}").format(history_df['val_loss'].min()))

In [ ]:
x_test = X_test.to_numpy()
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
prediction = model.predict(x_test)

In [ ]:
sample_sub = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
print(sample_sub.shape)
sample_sub.head()

In [ ]:
sample_sub['item_cnt_month'] = prediction
sample_sub.to_csv('prediction.csv', index=False)
sample_sub.head()